In [1]:
import pandas as pd
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFilter, ImageOps
import random
import os
import auto_maskmix_util
import matplotlib.pyplot as plt

In [42]:
save_dir_lab = "/home/tamaru/scene_categorize/main/data/dataset/augmented_image/lab"
save_dir_professorroom = "/home/tamaru/scene_categorize/main/data/dataset/augmented_image/professorroom"

In [3]:
lab_bg_mask = pd.read_csv("/home/tamaru/scene_categorize/main/data/dataset/lab_bg_mask.csv", names=["path", "data", "mask_cube"])
professorroom_bg_mask = pd.read_csv("/home/tamaru/scene_categorize/main/data/dataset/professorroom_bg_mask.csv", names=["path", "data", "mask_cube"])
lab_mix_img = pd.read_csv("/home/tamaru/scene_categorize/main/data/dataset/lab_mix_img.csv", names=["path", "data", "mix_img"])
professorroom_mix_img = pd.read_csv("/home/tamaru/scene_categorize/main/data/dataset/professorroom_mix_img.csv", names=["path", "data", "mix_img"])

In [4]:
lab_bg_mask = lab_bg_mask.drop(["data", "mask_cube"], axis=1)
lab_mix_img = lab_mix_img.drop(["data", "mix_img"], axis=1)
professorroom_bg_mask = professorroom_bg_mask.drop(["data", "mask_cube"], axis=1)
professroroom_mix_img = professorroom_mix_img.drop(["data", "mix_img"], axis=1)

In [5]:
lab_bg_mask

,path
0,/home/tamaru/scene_categorize/main/data/datase...
1,/home/tamaru/scene_categorize/main/data/datase...
2,/home/tamaru/scene_categorize/main/data/datase...
3,/home/tamaru/scene_categorize/main/data/datase...
4,/home/tamaru/scene_categorize/main/data/datase...
5,/home/tamaru/scene_categorize/main/data/datase...
6,/home/tamaru/scene_categorize/main/data/datase...
7,/home/tamaru/scene_categorize/main/data/datase...
8,/home/tamaru/scene_categorize/main/data/datase...
9,/home/tamaru/scene_categorize/main/data/datase...


In [6]:
lab_bg_mask = lab_bg_mask.sort_values("path")
lab_bg_mask = lab_bg_mask.reset_index(drop=True)
lab_bg_mask

,path
0,/home/tamaru/scene_categorize/main/data/datase...
1,/home/tamaru/scene_categorize/main/data/datase...
2,/home/tamaru/scene_categorize/main/data/datase...
3,/home/tamaru/scene_categorize/main/data/datase...
4,/home/tamaru/scene_categorize/main/data/datase...
5,/home/tamaru/scene_categorize/main/data/datase...
6,/home/tamaru/scene_categorize/main/data/datase...
7,/home/tamaru/scene_categorize/main/data/datase...
8,/home/tamaru/scene_categorize/main/data/datase...
9,/home/tamaru/scene_categorize/main/data/datase...


In [7]:
professorroom_bg_mask = professorroom_bg_mask.sort_values("path")
professorroom_bg_mask = professorroom_bg_mask.reset_index(drop=True)
professorroom_bg_mask

,path
0,/home/tamaru/scene_categorize/main/data/datase...
1,/home/tamaru/scene_categorize/main/data/datase...
2,/home/tamaru/scene_categorize/main/data/datase...
3,/home/tamaru/scene_categorize/main/data/datase...
4,/home/tamaru/scene_categorize/main/data/datase...
5,/home/tamaru/scene_categorize/main/data/datase...
6,/home/tamaru/scene_categorize/main/data/datase...
7,/home/tamaru/scene_categorize/main/data/datase...
8,/home/tamaru/scene_categorize/main/data/datase...
9,/home/tamaru/scene_categorize/main/data/datase...


In [8]:
lab_mix_img = lab_mix_img.sort_values("path")
lab_mix_img = lab_mix_img.reset_index(drop=True)
print(lab_mix_img.path[0])

/home/tamaru/scene_categorize/main/data/dataset/mix_img/lab/bottle.png


In [9]:
professorroom_mix_img = professorroom_mix_img.sort_values("path")
professorroom_mix_img = professorroom_mix_img.reset_index(drop=True)
print(professorroom_mix_img.path[0])

/home/tamaru/scene_categorize/main/data/dataset/mix_img/professorroom/insta360.png


In [10]:
background = pd.read_csv("/home/tamaru/scene_categorize/main/data/dataset/train.csv", names=["path", "data", "mask_cube"])
background = background.drop(["data"], axis=1)
lab_background = background[background.mask_cube=="lab"]
professorroom_background = background[background.mask_cube=="professorroom"]

In [11]:
lab_background = lab_background.drop(["mask_cube"], axis=1)
professorroom_background = professorroom_background.drop(["mask_cube"], axis=1)
lab_background = lab_background.sort_values("path")
lab_background = lab_background.reset_index(drop=True)
professorroom_background = professorroom_background.sort_values("path")
professorroom_background = professorroom_background.reset_index(drop=True)

In [12]:
lab_background

,path
0,/home/tamaru/scene_categorize/main/data/datase...
1,/home/tamaru/scene_categorize/main/data/datase...
2,/home/tamaru/scene_categorize/main/data/datase...
3,/home/tamaru/scene_categorize/main/data/datase...
4,/home/tamaru/scene_categorize/main/data/datase...
5,/home/tamaru/scene_categorize/main/data/datase...
6,/home/tamaru/scene_categorize/main/data/datase...
7,/home/tamaru/scene_categorize/main/data/datase...
8,/home/tamaru/scene_categorize/main/data/datase...
9,/home/tamaru/scene_categorize/main/data/datase...


def auto_maskmix(FG_IMG, BG_IMG, SPEC_BG_IMG, fg_mask):
    foreground = Image.open(FG_IMG)
    background = Image.open(BG_IMG)
    foreground = foreground.resize((256,256))
    background = background.resize((256,256))
    fg_mask = Image.open(fg_mask)
#     fg_mask = ImageOps.invert(fg_mask)
    mask_im = fg_mask.resize(foreground.size).convert("L")
    
    coord = auto_maskmix_util.foreground_random_sampling(BG_IMG, SPEC_BG_IMG)
    
    back_im = background.copy()
    x, y = coord
    rate = np.random.rand()*0.5 + 0.5
    rotate = random.randint(0,360)
    foreground_ = foreground.resize((int(foreground.width*rate), int(foreground.height*rate))).rotate(rotate)
    mask_im_ = mask_im.resize((int(mask_im.width*rate), int(mask_im.height*rate))).rotate(rotate)
    #前景画像の配置位置の座標は前景画像の左上なので，中心に平行移動する->未完成
    back_im.paste(foreground_, (int(x-foreground_.size[0]/2), int(y-foreground_.size[1]/2)), mask_im_)
    print("Foreground center coordinate", int(x-foreground_.size[0]/2), int(y-foreground_.size[1]/2))
    print("Foreground Rate:", rate)
    print("Foreground Size", foreground_.size[1])
    return back_im
    

def auto_maskmix(FG_IMG, BG_IMG, SPEC_BG_IMG):
    foreground = Image.open(FG_IMG)
    background = Image.open(BG_IMG)
    foreground = foreground.resize((256,256))
    background = background.resize((256,256))
    
    coord = auto_maskmix_util.foreground_random_sampling(BG_IMG, SPEC_BG_IMG)
    
    back_im = background.copy()
    x, y = coord
    rate = np.random.rand()*0.5 + 0.5
    rotate = random.randint(0,360)
    foreground_ = foreground.resize((int(foreground.width*rate), int(foreground.height*rate))).rotate(rotate)
    #前景画像の配置位置の座標は前景画像の左上なので，中心に平行移動する->未完成
    back_im.paste(foreground_, (int(x-foreground_.size[0]/2), int(y-foreground_.size[1]/2)))
    print("Foreground center coordinate", int(x-foreground_.size[0]/2), int(y-foreground_.size[1]/2))
    print("Foreground Rate:", rate)
    print("Foreground Size", foreground_.size[1])
    return back_im

In [37]:
class cv2OverlayImage(object):
    """
    [summary]
      Opencv2形式の画像に指定画像を重ねる
    """

    def __init__(self):
        pass

    @classmethod
    def overlay(
            cls,
            cv2_background_image,
            cv2_overlay_image,
            point,
    ):
        """
        [summary]
          Opencv2形式の画像に指定画像を重ねる
        Parameters
        ----------
        cv2_background_image : [Opencv2 Image]
        cv2_overlay_image : [Opencv2 Image]
        point : [(x, y)]
        Returns : [Opencv2 Image]
        """
        overlay_height, overlay_width = cv2_overlay_image.shape[:2]

        # Opencv2形式の画像をPIL形式に変換(α値含む)
        # 背景画像ユーザー
        cv2_rgb_bg_image = cv2.cvtColor(cv2_background_image, cv2.COLOR_BGR2RGB)
        pil_rgb_bg_image = Image.fromarray(cv2_rgb_bg_image)
        pil_rgba_bg_image = pil_rgb_bg_image.convert('RGBA')
        # オーバーレイ画像
        cv2_rgb_ol_image = cv2.cvtColor(cv2_overlay_image, cv2.COLOR_BGRA2RGBA)
        pil_rgb_ol_image = Image.fromarray(cv2_rgb_ol_image)
        pil_rgba_ol_image = pil_rgb_ol_image.convert('RGBA')
        # composite()は同サイズ画像同士が必須のため、合成用画像を用意
        pil_rgba_bg_temp = Image.new('RGBA', pil_rgba_bg_image.size,
                                     (255, 255, 255, 0))
        # 座標を指定し重ね合わせる
        pil_rgba_bg_temp.paste(pil_rgba_ol_image, point, pil_rgba_ol_image)
        result_image = \
            Image.alpha_composite(pil_rgba_bg_image, pil_rgba_bg_temp)

        # Opencv2形式画像へ変換
        cv2_bgr_result_image = cv2.cvtColor(
            np.asarray(result_image), cv2.COLOR_RGBA2BGRA)

        return cv2_bgr_result_image

In [44]:
def auto_maskmix(FG_IMG, BG_IMG, SPEC_BG_IMG):
    foreground = cv2.imread(FG_IMG, cv2.IMREAD_UNCHANGED)
    background = cv2.imread(BG_IMG)
    foreground = cv2.resize(foreground, (256,256))
    background = cv2.resize(background, (256,256))
    
    coord = auto_maskmix_util.foreground_random_sampling(BG_IMG, SPEC_BG_IMG)
    
    back_im = background.copy()
    x, y = coord
    rate = np.random.rand()*0.2 + 0.2
    rotate = random.randint(0,360)
    size = (int(foreground.shape[1]*rate), int(foreground.shape[1]*rate))
    foreground_ = cv2.resize(foreground, size)
    #前景画像の配置位置の座標は前景画像の左上なので，中心に平行移動する->未完成
    point = (int(x-foreground_.shape[1]/2), int(y-foreground_.shape[0]/2))
    back_im = cv2OverlayImage.overlay(background ,foreground_, point)
    print("Foreground center coordinate", int(x-foreground_.shape[0]/2), int(y-foreground_.shape[1]/2))
    print("Foreground Rate:", rate)
    print("Foreground Size", foreground_.shape[1])
    return back_im

In [45]:
def auto_maskmix_2all(background, bg_mask, mix_img, save_dir):
    bg_col_name = background.columns.values
    bg_mask_col_name = bg_mask.columns.values
    mix_col_name = mix_img.columns.values
    background_np = np.array(background)
    bg_mask_np = np.array(bg_mask)
    mix_img_np = np.array(mix_img)
    i = 0
    while(background_np[i][bg_col_name.tolist().index("path")] is not None):
        rand_mix = random.randint(0,len(mix_img_np)-1)
        print("Foreground:", rand_mix)
        back_im = auto_maskmix(mix_img_np[rand_mix][0], background_np[i][bg_col_name.tolist().index("path")], bg_mask_np[i][bg_mask_col_name.tolist().index("path")])
        cv2.imwrite(save_dir+"/"+background_np[i][bg_col_name.tolist().index("path")].split('/')[-1], back_im)
        i += 1
        print("------------")
    

In [46]:
auto_maskmix_2all(lab_background,lab_bg_mask,lab_mix_img, save_dir_lab)

Foreground: 3
Foreground center coordinate 177 108
Foreground Rate: 0.20856159433639065
Foreground Size 53
------------
Foreground: 3
Foreground center coordinate 170 21
Foreground Rate: 0.3520508917550733
Foreground Size 90
------------
Foreground: 2
Foreground center coordinate 119 90
Foreground Rate: 0.2628855656176739
Foreground Size 67
------------
Foreground: 3
Foreground center coordinate 107 148
Foreground Rate: 0.2608647077693254
Foreground Size 66
------------
Foreground: 0
Foreground center coordinate 166 17
Foreground Rate: 0.3603160398913633
Foreground Size 92
------------
Foreground: 0
Foreground center coordinate 118 45
Foreground Rate: 0.23842889055836425
Foreground Size 61
------------
Foreground: 2
Foreground center coordinate 143 82
Foreground Rate: 0.36096037395161357
Foreground Size 92
------------
Foreground: 1
Foreground center coordinate 134 194
Foreground Rate: 0.3571341065125248
Foreground Size 91
------------
Foreground: 2
Foreground center coordinate 221 130

ValueError: 'a' cannot be empty unless no samples are taken

In [48]:
auto_maskmix_2all(professorroom_background,professorroom_bg_mask,professorroom_mix_img, save_dir_professorroom)

Foreground: 2
Foreground center coordinate 123 117
Foreground Rate: 0.30103511652485426
Foreground Size 77
------------
Foreground: 1
Foreground center coordinate 187 -36
Foreground Rate: 0.35715498022384073
Foreground Size 91
------------
Foreground: 0
Foreground center coordinate 169 5
Foreground Rate: 0.2862701229403325
Foreground Size 73
------------
Foreground: 3
Foreground center coordinate 218 -7
Foreground Rate: 0.24349361387795254
Foreground Size 62
------------
Foreground: 0
Foreground center coordinate 226 199
Foreground Rate: 0.22123206905734547
Foreground Size 56
------------
Foreground: 2
Foreground center coordinate 196 192
Foreground Rate: 0.24063199636412202
Foreground Size 61
------------
Foreground: 0
Foreground center coordinate 194 65
Foreground Rate: 0.29350303998280103
Foreground Size 75
------------
Foreground: 3
Foreground center coordinate 201 67
Foreground Rate: 0.33985322741441737
Foreground Size 87
------------
Foreground: 0
Foreground center coordinate 63 

IndexError: index 20 is out of bounds for axis 0 with size 20

mix_img_ = pd.read_csv("/home/tamaru/scene_categorize/main/data/mix_img.csv", names=["path", "data", "mix_img"])
mix_img_ = mix_img_.drop(["data", "mix_img"], axis=1)

def mask_with_grabcut(FG_IMG):
    bgr = cv2.imread(FG_IMG)
    bgr = cv2.cvtColor(bgr, cv2.COLOR_RGB2BGR)
    h, w = bgr.shape[:2]
    mask = np.zeros((h,w), dtype = np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect=(1,1,w,h)
    cv2.grabCut(bgr, mask, rect, bgdModel, fgdModel, 10, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    bgr2 = bgr*mask2[:,:,np.newaxis]
    bgr2 = cv2.resize(bgr2, (256,256))
    # 輪郭抽出する。
    contours, _ = cv2.findContours(bgr2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # 検出された輪郭内部を255で塗りつぶす。
    mask = np.zeros_like(bgr2)
    cv2.drawContours(mask, contours, -1, color=(255,255,255), thickness=-1)

    # 以上の手順で前景画像のうち、合成する画素を255としたマスク画像が作成できた。
    mask = cv2.resize(mask, (256,256))
    return mask


mix_img_np = np.array(mix_img_)
for row in mix_img_np: 
    print(row[0])
    print(row[0].split("/")[-1])
    mask = mask_with_grabcut(row[0])
    cv2.imwrite("/home/tamaru/scene_categorize/main/data/mask/"+row[0].split("/")[-1],mask)

mix_img_np = np.array(mix_img)
for row in mix_img_np: 
    print(row[0])
    print(row[0].split("/")[-1])
    image = cv2.imread(row[0], cv2.IMREAD_GRAYSCALE)
    # 2値化する。
    _, binary = cv2.threshold(image, 10, 255, cv2.THRESH_TOZERO)
    # 輪郭抽出する。
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # 検出された輪郭内部を255で塗りつぶす。
    mask = np.zeros_like(image)
    cv2.drawContours(mask, contours, -1, color=(255,255,255), thickness=-1)
    mask =cv2.resize(mask, (256,256))
    cv2.imwrite("/home/tamaru/scene_categorize/main/data/mask/"+row[0].split("/")[-1],mask)